In [1]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join

Utilize a função a abaixo para segmentar o rosto de uma imagem.

In [2]:
face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)
    
    if faces is ():
        return None, None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face, faces

### Captura de faces para uso nos algoritmos Eigenfaces, Fisherfaces e LBPH

In [54]:
cap = cv2.VideoCapture(0)

contagem = 0

contagem_maxima = 200

while True:
    ret, frame = cap.read()

    if ret:
        cv2.imshow("Imagem de Treino", frame)
        
        face, faces = face_extractor(frame)
        if face is not None:
            face_norm = cv2.resize(face, (200, 200))
            face_norm = cv2.cvtColor(face_norm, cv2.COLOR_BGR2GRAY)
            file_name_path = "imagens/train/a/"+str(contagem)+".png"
            cv2.imwrite(file_name_path, face_norm)
            contagem += 1
        # Se for teclado Enter (tecla 13) deverá sair do loop e encerrar a captura de imagem
        # ou for alcançado a contagem máxima (amostras)
        if cv2.waitKey(1) == 13 or contagem == contagem_maxima:
            break
        
cap.release()
cv2.destroyAllWindows()
print("Coleta de amostras completado")

Coleta de amostras completado


### Leitura das imagens para treinamento

In [61]:
data_path_a = "imagens/train/a/"
data_path_b = "imagens/train/b/"

training_data, labels = [], []

onlyfiles_a = [f for f in listdir(data_path_a) if isfile(join(data_path_a, f))]
onlyfiles_b = [f for f in listdir(data_path_b) if isfile(join(data_path_b, f))]

for i, files in enumerate(onlyfiles_a):
    image_path = data_path_a + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    training_data.append(images)
    labels.append(0)

for i, files in enumerate(onlyfiles_b):
    image_path = data_path_b + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    training_data.append(images)
    labels.append(1)
    
# Criando uma matriz da lista de labels
labels = np.asarray(labels, dtype=np.int32)

person = {0: "TESTE 01", 1: "TESTE 02"}

### Eigenfaces

In [62]:
# Treinamento do modelo
modelEigenFace = cv2.face.EigenFaceRecognizer_create()
modelEigenFace.train(training_data, labels)

print("Modelo Eigen Face treinado com sucesso.")

Modelo Eigen Face treinado com sucesso.


### Fisherfaces

In [63]:
# Treinamento do modelo
modelFisherFace = cv2.face.FisherFaceRecognizer_create()
modelFisherFace.train(training_data, labels)

print("Modelo Fisher Face treinado com sucesso.")

Modelo Fisher Face treinado com sucesso.


### LBPH - Local Binary Patterns Histograms

In [65]:
# Treinamento do modelo
modelLBPH = cv2.face.LBPHFaceRecognizer_create()
modelLBPH.train(training_data, labels)

print("Modelo LBPH treinado com sucesso.")

Modelo LBPH treinado com sucesso.


### Aplicando os modelos

In [66]:
cap = cv2.VideoCapture(0)

total = 0
successo = 0
nao_identificado = 0
no_face = 0
img_capturada = False

while True:
    ret, frame = cap.read()
    
    if ret:
        
        total += 1
        
        # Face da imagem obtida da câmera
        face, faces = face_extractor(frame)
        
        if face is None:
            cv2.putText(frame, "FACE AUSENTE", (25,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        else:
            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            
            face_norm = cv2.resize(face, (200, 200))
            face_norm = cv2.cvtColor(face_norm, cv2.COLOR_BGR2GRAY)
            
            resultsEigenFace = modelEigenFace.predict(face_norm)
            resultsFisherFace = modelFisherFace.predict(face_norm)
            resultsLBPH = modelLBPH.predict(face_norm)
            
            infoEigenFace = "EigenFace: " + str(int(resultsEigenFace[1]/100)) + " pontos"
            infoFisherFace = "FisherFace: " + str(int(resultsFisherFace[1]/10)) + " pontos"
            infoLBPH = "LBPH: " + str(int(resultsLBPH[1])) + " pontos"
            
            cv2.putText(frame, infoEigenFace, (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
            cv2.putText(frame, infoFisherFace, (25, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
            cv2.putText(frame, infoLBPH, (25, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
            
            if (resultsEigenFace[1] >= 3000 and resultsEigenFace[1] <= 4000):
                cv2.putText(frame, person[0] + " Reconhecido", (280, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
                
            if (resultsFisherFace[1] >= 300 and resultsFisherFace[1] <= 400):
                cv2.putText(frame, person[0] + " Reconhecido", (280, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
                
            if (resultsLBPH[1] >= 30 and resultsLBPH[1] <= 40):
                cv2.putText(frame, person[0] + " Reconhecido", (280, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
                if (not img_capturada):
                    cv2.imwrite("imagens/evidencias/face_reconhecida.png", frame)
                    img_capturada = True

        cv2.imshow("Reconhecimento de face", frame)
            
    # Se for teclado Enter (tecla 13) deverá sair do loop e encerrar a captura de imagem    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

print("Total de faces: " + str(total))

cap.release()
cv2.destroyAllWindows()

Total de faces: 410


In [10]:
cap.release()
cv2.destroyAllWindows()